### Le code est mis de coté pour cause du temps de calcul. Il s'intégre dans la partie train du modèle en remplacement des cellules de la partie Train.

In [ ]:
# LIGNE TO INSTALL THE PACKAGE
#pip install bayesian-optimization

____


In [ ]:
# Let's start by definying our function, bounds, and instanciating an optimization object.
def black_box_function(EMBEDDING_SIZE, SEED, HIDDEN_SIZE, N_LAYERS, LR, EPOCHS):

    torch.manual_seed(SEED) # set the random seed manually for reproducibility.
    pass
    
    # Display some figures for model dimensioning
    n_words = len(corpus.dictionary.word2idx)
    n_phons = len(corpus.dictionary_phon.word2idx)
    print(f"Number of words in the corpus: {n_words}.")
    print(f"Number of phonems in the corpus: {n_phons}.")
    print(f"Ratio [phonems / words]: {n_phons/n_words:.2f}.")
    log_words = np.log(n_words)
    log_phons = np.log(n_phons)
    print(f"Ratio [log(phonems) / log(words)]: {log_phons/log_words:.2f}.")
    
    # Define weights for the Cross Entropy Loss
    # The more frequent the word is, the less weighted 
    def get_weight(corpus):
        n_eos = int((corpus.train == 0).sum()) + int((corpus.test == 0).sum()) \
        + int((corpus.valid == 0).sum())
        weight = [n_eos]
        weight += sorted(list(corpus.dictionary.frequency.values()), reverse=True)
        weight = 1 / np.log(np.array(weight))
        return torch.tensor(weight, dtype=torch.float).to(device)
    
    # Loss selection
    WEIGHT = False
    if WEIGHT:
        weight = get_weight(corpus)
        criterion = nn.CrossEntropyLoss(weight=weight)
    else:
        criterion = nn.CrossEntropyLoss()

    # Model selection
    model = LSTMModel(ntoken=N_TOKENS, ninp=EMBEDDING_SIZE,
                    nhid=HIDDEN_SIZE, nlayers=N_LAYERS).to(device)


    n_params = torch.nn.utils.parameters_to_vector(model.parameters()).numel()
  
    
    
    OPTIMIZER = 'sgd'   # Optimizer (SGD or Adam)
    WDECAY = 0          # Weight decay
    CLIP = 0.25         # Gradient clippin
    
    if OPTIMIZER == 'sgd':
        optim = torch.optim.SGD(model.parameters(), lr=LR, weight_decay=WDECAY)
    elif OPTIMIZER == 'adam':
        optim = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WDECAY)
    else:
        optim = None

    if OPTIMIZER in ['sgd', 'adam']:
        scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=1, gamma=0.7)
    else:
        scheduler = None


    MODEL_NAME = 'model_classic'
    SAVE_PATH = os.path.join(CHECKPOINT_PATH, '{}.pt'.format(MODEL_NAME))
    LOAD_EXISTING_MODEL = False  # Change to False if you want to start from scratch
                                # ⚠ If False, any existing model with the same name
                                # will be overwritten ⚠
    TENSORBOARD = True           # Monitor the training


    if LOAD_EXISTING_MODEL:
        try:
            with open(SAVE_PATH, 'rb') as f:
                model = torch.load(f)
                # after load the rnn params are not a continuous chunk of memory
                # this makes them a continuous chunk, and will speed up forward pass
                model.rnn.flatten_parameters()
            print("Successfully loaded model from {}".format(SAVE_PATH))
        except:
            pass
        
    best_val_loss = None

    # Loop over epochs.
    # At any point you can hit Ctrl + C to break out of training early.
    try:

        train_data, val_data, test_data = get_train_valid_test_data(model.phon,
                                                                    ordered=True)
        # Initialize tensorboard
        if TENSORBOARD:
            iter = 1
            comment=f'model={MODEL_NAME}_IN={INPUT_SIZE}_INPHON={INPUT_SIZE_PHON}'
            writer = SummaryWriter(comment=comment)
            for name, weight in model.named_parameters():
                writer.add_histogram(name, weight, 0)

        for epoch in range(1, EPOCHS+1):
            epoch_start_time = time.time()
            train()
            val_loss = min(20, evaluate(val_data))
            if scheduler is not None:
                scheduler.step()
            # Monitoring
            if TENSORBOARD:
                for name, weight in model.named_parameters():
                    writer.add_histogram(name, weight, epoch)
                    writer.add_histogram(f'{name}.grad', weight.grad, epoch)
                writer.add_scalar('Loss_Val', val_loss, epoch)

            # Save the model if the validation loss is the best we've seen so far.
            if not best_val_loss or val_loss < best_val_loss:
                with open(SAVE_PATH, 'wb') as f:
                    torch.save(model, f)
                print("Successfully saved model at {}\n".format(SAVE_PATH))
                best_val_loss = val_loss
            else:
                if scheduler is None:
                    lr /= 4.0

        if TENSORBOARD:
            writer.flush()
            writer.close()

    except KeyboardInterrupt:
        print('-' * 89)
        print('Exiting from training early')
        if TENSORBOARD:
            writer.flush()
            writer.close()

    
    test_loss = evaluate(test_data)


    return -test_loss

In [ ]:
# Bounded region of parameter space
pbounds = {'SEED': (30, 50), 'EMBEDDING_SIZE': (250, 1250), 'HIDDEN_SIZE': (500, 2000), 'N_LAYERS': (10,30) , 'LR' : (0,20), 'LR' : (5,10)  }

In [ ]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [ ]:
optimizer.maximize(
    init_points=2,
    n_iter=5,
    acq="ei", 
    xi=1e-4
)

In [ ]:
print(optimizer.max)